<a href="https://colab.research.google.com/github/jschuller/zoomcamp-stock-market-analytics/blob/main/HW_Module01_Colab_Introduction_and_Data_Sources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework - Module 1 Stock Invest Zoomcamp - April 2024

In [ ]:
# install main library YFinance
!pip install yfinance

In [99]:
# import libraries
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date


In [29]:
# set a default index period
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=21
Period for indexes: 1954-04-21 to 2024-04-21 


# Question 1: What is the average growth (in %) of GDP in 2023?
*   Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1).
*   Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago).
*   Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth


In [31]:
# Real Gross Domestic Product (GDPC1), Billions of Chained 2017 Dollars,
# Seasonally Adjusted Annual Rate, QUARTERLY
# https://fred.stlouisfed.org/series/GDPC1
gdppc1 = pdr.DataReader('GDPC1', 'fred', start=start)

In [32]:
gdppc1['gdppc1_us_yoy'] = gdppc1.GDPC1/gdppc1.GDPC1.shift(4)-1
gdppc1['gdppc1_us_qoq'] = gdppc1.GDPC1/gdppc1.GDPC1.shift(1)-1
gdppc1.tail()

,GDPC1,gdppc1_us_yoy,gdppc1_us_qoq
DATE,,,
2022-10-01,21989.981,0.006517,0.006354
2023-01-01,22112.329,0.017179,0.005564
2023-04-01,22225.350,0.023825,0.005111
2023-07-01,22490.692,0.029269,0.011939
2023-10-01,22679.255,0.031345,0.008384


In [33]:
# Find the average YoY growth in 2023 (average from 4 YoY numbers).
# Round to 1 digit after the decimal point: e.g. if you get 5.66% growth

# Find the average YoY growth in 2023 (average from 4 YoY numbers).
average_growth_2023 = round(gdppc1.loc['2023-01-01':'2023-12-31', 'gdppc1_us_yoy'].mean() * 100, 1)

# Print the average growth
print(f'Average GDP growth in 2023: {average_growth_2023}%')


Average GDP growth in 2023: 2.5%


#  Question 2. [Macro] Inverse "Treasury Yield"
The Treasury yield curve is a crucial tool in the world of finance and economics. It represents the relationship between the yield (interest rate) and the maturity of a set of similar debt instruments, such as government bonds. The difference between the 10-year and 2-year Treasury yields, often referred to as the "yield spread," is an important indicator of economic conditions and investor sentiment.
*   Find the min value of (dgs10-dgs2) since year 2000 (2000-01-01)
*   Write it down as an answer and round to 1 digit after the decimal point
*   Download DGS2 and DGS10 interest rates series
    *  https://fred.stlouisfed.org/series/DGS2
    *  https://fred.stlouisfed.org/series/DGS10
*   Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join())
*   Calculate the difference dgs10-dgs2 daily.


In [40]:
# REWORK
import pandas_datareader as pdr

# Download the 2-year and 10-year Treasury yields from the FRED website
# since year 2000 (2000-01-01)
start='2000-01-01'
dgs2 = pdr.DataReader('DGS2', 'fred', start=start)
dgs10 = pdr.DataReader('DGS10', 'fred', start=start)

# Merge the data into a single DataFrame using the merge() method, making sure
# to align the data on the DATE column
df = dgs2.merge(dgs10, on='DATE', how='inner')

# Calculate the difference between the 10-year and 2-year yields
df['yield_curve_spread'] = df['DGS10'] - df['DGS2']

# Find the minimum value of the yield curve spread
min_spread = round(df['yield_curve_spread'].min(),1)
print(f'The minimum value of the yield curve spread since 2000 is {min_spread}.')




The minimum value of the yield curve spread since 2000 is -1.1.


# Question 3. [Index] Which Index is better recently?
*   Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)
*   Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/).
*   Compare 5Y growth for both (between 2019-04-09 and 2024-04-09).
*   Select the higher growing index and write down the growth in %  (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.


In [44]:
# let's the Yahoo Finance API to download the daily price data for these two indexes
import yfinance as yf

# Download S&P 500 data
sp500 = yf.download('^GSPC', start='2019-04-09', end='2024-04-09')

# Download IPC Mexico data
ipc_mexico = yf.download('^MXX', start='2019-04-09', end='2024-04-09')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [43]:
# Calculate 5-year growth for S&P 500
sp500_start = sp500['Close'].iloc[0]
sp500_end = sp500['Close'].iloc[-1]
sp500_growth = (sp500_end / sp500_start) - 1
print(f"S&P 500 5-year growth: {sp500_growth * 100:.0f}%")

# Calculate 5-year growth for IPC Mexico
ipc_mexico_start = ipc_mexico['Close'].iloc[0]
ipc_mexico_end = ipc_mexico['Close'].iloc[-1]
ipc_mexico_growth = (ipc_mexico_end / ipc_mexico_start) - 1
print(f"IPC Mexico 5-year growth: {ipc_mexico_growth * 100:.0f}%")

S&P 500 5-year growth: 81%
IPC Mexico 5-year growth: 28%


# Question 4. [Stocks OHLCV] 52-week range ratio (2023) for the selected stocks
First, let's set the stage by understanding the 52-week range ratio. This metric is calculated as the difference between the maximum and minimum adjusted closing prices for a stock over a 52-week (one year) period, divided by the maximum adjusted closing price. This gives us a sense of how much the stock's price has fluctuated within that timeframe, with a higher ratio indicating a larger price range.
*   Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023
*   Download the 2023 daily OHLCV data on Yahoo Finance for top5 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.
*   Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history
*   Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)

In [45]:
# We'll be using the Yahoo Finance API to download the 2023 daily OHLCV (Open, High, Low, Close, Volume)
# data for each of these stocks: 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM

import pandas as pd

# List of tickers
tickers = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

# Create an empty dictionary to store the results
results = {}

# Loop through the tickers and calculate the 52-week range ratio
for ticker in tickers:
    # Download the 2023 daily OHLCV data for the current ticker
    df = pd.read_csv(f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1=1672531200&period2=1704067200&interval=1d&events=history&includeAdjustedClose=true')

    # Convert the 'Date' column to a datetime object
    df['Date'] = pd.to_datetime(df['Date'])

    # Filter the data to only include 2023
    df = df[df['Date'].dt.year == 2023]

    # Calculate the 52-week range ratio
    max_price = df['Adj Close'].max()
    min_price = df['Adj Close'].min()
    range_ratio = round((max_price - min_price) / max_price, 2)

    # Store the result in the dictionary
    results[ticker] = range_ratio

# Print the results
for ticker, ratio in results.items():
    print(f"The 52-week range ratio for {ticker} in 2023 is: {ratio}")



The 52-week range ratio for 2222.SR in 2023 is: 0.21
The 52-week range ratio for BRK-B in 2023 is: 0.21
The 52-week range ratio for AAPL in 2023 is: 0.37
The 52-week range ratio for MSFT in 2023 is: 0.42
The 52-week range ratio for GOOG in 2023 is: 0.39
The 52-week range ratio for JPM in 2023 is: 0.28


# Question 5. [Stocks] Dividend Yield
*   Find the largest dividend yield for the same set of stocks
*   Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023. You can use get_actions() method or .dividends field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)
*   Sum up all dividends paid in 2023 per company

In [71]:
# First, let's import the necessary libraries and get our list of companies.  we'll use the yfinance library to download the
# dividend data for each company.
import yfinance as yf
tickers = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

In [72]:
#  Next, we'll store the total dividends paid in 2023 for each company in a dictionary.
dividends = {}

for ticker in tickers:
    company = yf.Ticker(ticker)
    total_dividends = company.dividends['2023'].sum()
    dividends[ticker] = total_dividends

In [73]:
# Let's calculate the dividend yield for each company and find the largest one.
prices = {ticker: yf.Ticker(ticker).info['currentPrice'] for ticker in tickers}
dividend_yields = {ticker: (dividends[ticker] / prices[ticker]) * 100 for ticker in tickers}
largest_yield = max(dividend_yields.values())
best_stock = [k for k, v in dividend_yields.items() if v == largest_yield][0]

print(f"The largest dividend yield in 2023 is {largest_yield:.2f}% for {best_stock}.")

The largest dividend yield in 2023 is 3.03% for 2222.SR.


In [74]:
# Print the dividend yields for each stock
for ticker, dividend_yield in dividend_yields.items():
    print(f"Dividend yield for {ticker}: {dividend_yield:.2f}%")

Dividend yield for 2222.SR: 3.03%
Dividend yield for BRK-B: 0.00%
Dividend yield for AAPL: 0.58%
Dividend yield for MSFT: 0.70%
Dividend yield for GOOG: 0.00%
Dividend yield for JPM: 2.18%


# Question 6. [Exploratory] Investigate new metrics
*   Free text answer
*   Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).
*   Download data on the number of people employed in the United States

In [87]:
# These metrics can be used to track the performance of the economy and to make investment decisions.
import pandas_datareader as pdr
start='2023-04-01'

# The unemployment rate is a key indicator of the health of the economy.
# Fetch the US employment data from FRED
emp_data = pdr.DataReader('UNRATE', 'fred', start=start)

# The consumer price index is a measure of inflation.
# Download data on the consumer price index from FRED
cpi = pdr.get_data_fred('CPIAUCSL', start=start)

# The S&P 500 index is a measure of the stock market.
# Download data on the S&P 500 index from yFinance
sp500 = yf.download('^GSPC', start=start)

# Take a quick look at the data
print(emp_data.head(12))
print(cpi.head(12))
print(sp500.head(12))



[*********************100%%**********************]  1 of 1 completed

            UNRATE
DATE              
2023-04-01     3.4
2023-05-01     3.7
2023-06-01     3.6
2023-07-01     3.5
2023-08-01     3.8
2023-09-01     3.8
2023-10-01     3.8
2023-11-01     3.7
2023-12-01     3.7
2024-01-01     3.7
2024-02-01     3.9
2024-03-01     3.8
            CPIAUCSL
DATE                
2023-04-01   303.032
2023-05-01   303.365
2023-06-01   304.003
2023-07-01   304.628
2023-08-01   306.187
2023-09-01   307.288
2023-10-01   307.531
2023-11-01   308.024
2023-12-01   308.742
2024-01-01   309.685
2024-02-01   311.054
2024-03-01   312.230
                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2023-04-03  4102.200195  4127.660156  4098.790039  4124.509766  4124.509766   
2023-04-04  4128.029785  4133.129883  4086.870117  4100.600098  4100.600098   
2023-04-05  4094.500000  4099.689941  4072.560059  4090.379883  4090.379883   
2023-04-06  4081.149902  4107.319824  4069.84

# Question 7. [Exploratory] Time-driven strategy description around earnings releases
*   Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar (https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23).
*   Compare with the previous closed earnings (e.g., recent dates with full data https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08).
*   Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.

By comparing the upcoming earnings dates with the recent closed earnings dates, we can start to identify patterns and potential opportunities. For example, we could look for companies that have consistently beaten or missed their earnings expectations, or we could focus on industries or sectors that are showing significant activity around earnings releases.

To develop an analytical strategy, we could use machine learning techniques to create a model that predicts the market's reaction to a company's earnings report. This could involve training the model on historical data, such as stock price movements, earnings surprises, and other relevant features. Then, we could use the model to evaluate the upcoming earnings releases and identify a subset of companies that are likely to experience a significant price movement.

Here's a simple example of how we could use a linear regression model to predict a company's stock price movement based on its earnings surprise:

In [90]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Assume we have historical data on earnings surprises and stock price movements
earnings_surprises = [0.05, -0.02, 0.08, -0.03, 0.01]
stock_price_movements = [0.08, -0.04, 0.12, -0.07, 0.03]

# Train the linear regression model
model = LinearRegression()
model.fit(np.array(earnings_surprises).reshape(-1, 1), np.array(stock_price_movements))

# Use the model to predict the stock price movement for an upcoming earnings release
upcoming_earnings_surprise = 0.06
predicted_stock_price_movement = model.predict([[upcoming_earnings_surprise]])[0]
print(f"Predicted stock price movement: {predicted_stock_price_movement:.2f}")

Predicted stock price movement: 0.09


This is just a simple example, but you could build more complex models that take into account additional factors, such as market sentiment, industry trends, and macroeconomic conditions.